In [1]:
import os
from glob import glob

import geopandas as gpd

from agrilearn.crop_classification.config import DATA_REQUIREMENTS
from agrilearn.crop_classification.populate_eopatch_with_training_data import (
    PopulateEoPatchWithTrainingData,
)

2024-11-19 17:32:32.469183: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 17:32:33.521802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATASET_PATH = "/agrilearn_app/datasets/culture_v02"

NEW_DATASET_PATH = "/agrilearn_app/datasets/culture_v02_rnn_model"

CROPS = ["corn", "cotton", "rice", "soybean"]
SETS = ["train", "val", "test"]

In [ ]:

for crop in CROPS:
    for set_ in SETS:
        
        print(f"processing {crop} - {set_}")
        eopatch_folder = os.path.join(DATASET_PATH, crop, set_)
        eopatch_new_folder = os.path.join(NEW_DATASET_PATH, crop, set_)

        if not os.path.isdir(eopatch_new_folder):
            os.makedirs(eopatch_new_folder)


        gpkg_path = os.path.join(DATASET_PATH, f'{crop}_{set_}_v2.gpkg')
        # um gpkg for test, train, val
        gpkg = gpd.read_file(gpkg_path)

        gpkg.rename(columns={"monitoring_class": "crop_class"}, inplace=True)
        gpkg.crop_class = crop

        # gpkg_crop = gpkg_crop.iloc[:100]

        populate_eopatch = PopulateEoPatchWithTrainingData(
            eopatch_folder,
            DATA_REQUIREMENTS.TIMESTAMP_SEQ_SIZE,
            eopatch_new_folder,
            DATA_REQUIREMENTS.INTERP_DAY_RANGE,
        )

        _ = populate_eopatch.populate(
            gpkg, workers=8, save_report=True, remove_failed_samples=True
        )